# 这里先演示基础的一些步骤，辅助理解

In [1]:
import numpy as np
import pandas as pd
dtype = [("userId", np.int32), ("movieId", np.int32), ("rating", np.float32)]
dataset = pd.read_csv("../datasets/ml-latest-small/ratings.csv", usecols=range(3), dtype=dict(dtype))

In [2]:
dataset.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [3]:
users_ratings = dataset.groupby('userId').agg([list])
items_ratings = dataset.groupby('movieId').agg([list])

In [4]:
users_ratings.head()

,movieId,rating
,list,list
userId,,
1,"[1, 3, 6, 47, 50, 70, 101, 110, 151, 157, 163,...","[4.0, 4.0, 4.0, 5.0, 5.0, 3.0, 5.0, 4.0, 5.0, ..."
2,"[318, 333, 1704, 3578, 6874, 8798, 46970, 4851...","[3.0, 4.0, 4.5, 4.0, 4.0, 3.5, 4.0, 4.0, 4.5, ..."
3,"[31, 527, 647, 688, 720, 849, 914, 1093, 1124,...","[0.5, 0.5, 0.5, 0.5, 0.5, 5.0, 0.5, 0.5, 0.5, ..."
4,"[21, 32, 45, 47, 52, 58, 106, 125, 126, 162, 1...","[3.0, 2.0, 3.0, 2.0, 3.0, 3.0, 4.0, 5.0, 1.0, ..."
5,"[1, 21, 34, 36, 39, 50, 58, 110, 150, 153, 232...","[4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 5.0, 4.0, 3.0, ..."


In [5]:
items_ratings.head()

,userId,rating
,list,list
movieId,,
1,"[1, 5, 7, 15, 17, 18, 19, 21, 27, 31, 32, 33, ...","[4.0, 4.0, 4.5, 2.5, 4.5, 3.5, 4.0, 3.5, 3.0, ..."
2,"[6, 8, 18, 19, 20, 21, 27, 51, 62, 68, 82, 91,...","[4.0, 4.0, 3.0, 3.0, 3.0, 3.5, 4.0, 4.5, 4.0, ..."
3,"[1, 6, 19, 32, 42, 43, 44, 51, 58, 64, 68, 91,...","[4.0, 5.0, 3.0, 3.0, 4.0, 5.0, 3.0, 4.0, 3.0, ..."
4,"[6, 14, 84, 162, 262, 411, 600]","[3.0, 3.0, 3.0, 3.0, 1.0, 2.0, 1.5]"
5,"[6, 31, 43, 45, 58, 66, 68, 84, 103, 107, 111,...","[5.0, 3.0, 5.0, 3.0, 4.0, 4.0, 2.0, 3.0, 4.0, ..."


In [6]:
#初始化矩阵P和Q ，随机生成的,这里的10就是k
P = dict(zip(users_ratings.index,np.random.rand(len(users_ratings),10).astype(np.float32)))
P[2]

array([0.18535745, 0.84641695, 0.28204665, 0.42117122, 0.9200389 ,
       0.47364473, 0.6492416 , 0.013605  , 0.45307726, 0.94494534],
      dtype=float32)

In [7]:
#Q内有9724个物品，每个物品有10个特征
Q = dict(zip(items_ratings.index,np.random.rand(len(items_ratings),10).astype(np.float32)))
Q[1]

array([0.30661684, 0.5393968 , 0.7715378 , 0.09884772, 0.51083106,
       0.74712837, 0.439949  , 0.3295462 , 0.9290948 , 0.5435008 ],
      dtype=float32)

In [8]:
len(Q)

9724

In [9]:
#拿了第一个P,Q，这里没有想乘，实际是想乘得到
for uid,iid,real_rating in dataset.itertuples(index = False):
    print(uid,iid)
    print(P[uid],Q[iid])
    break

1 1
[9.2145336e-01 7.4461687e-01 6.8314296e-01 8.9935601e-01 9.5930237e-01
 2.1264568e-01 8.4361818e-04 1.1229287e-01 3.1255949e-01 3.5884235e-02] [0.30661684 0.5393968  0.7715378  0.09884772 0.51083106 0.74712837
 0.439949   0.3295462  0.9290948  0.5435008 ]


In [10]:
#向量点积
np.dot([1,2,3],[4,5,6])

32

In [11]:
#梯度下降优化损失函数
for i in range(15):#epoch 15次
    print('*'*10,i)
    for uid,iid,real_rating in dataset.itertuples(index = False):
        #遍历 用户 物品的评分数据 通过用户的id 到用户矩阵中获取用户向量
        v_puk = P[uid]
        # 通过物品的uid 到物品矩阵里获取物品向量
        v_qik = Q[iid]
        #计算损失，dot就是两个向量的点积，点积就是想乘并求和
        error = real_rating-np.dot(v_puk,v_qik)
        # 0.02学习率 0.01正则化系数
        v_puk += 0.02*(error*v_qik-0.01*v_puk)
        v_qik += 0.02*(error*v_puk-0.01*v_qik)
        
        P[uid] = v_puk #更新完毕后，放回字典
        Q[iid] = v_qik

********** 0
********** 1
********** 2
********** 3
********** 4
********** 5
********** 6
********** 7
********** 8
********** 9
********** 10
********** 11
********** 12
********** 13
********** 14


In [12]:
def predict_2(uid,iid):
    if uid not in users_ratings.index or iid not in items_ratings.index:
        raise Exception("uid or iid not in dataset")
    p_u= P[uid]
    q_i= Q[iid]
    return np.dot(p_u,q_i)

In [13]:
predict_2(1,1)

4.4670577

# 开始实现LFM模型

In [14]:
#数据集的拆分，针对用户，进行了针对性的优化
def data_split(data_path, x=0.8, random=False):
    '''
    切分数据集， 这里为了保证用户数量保持不变，将每个用户的评分数据按比例进行拆分
    :param data_path: 数据集路径
    :param x: 训练集的比例，如x=0.8，则0.2是测试集
    :param random: 是否随机切分，默认False
    :return: 用户-物品评分矩阵
    '''
    print("开始切分数据集...")
    # 设置要加载的数据字段的类型
    dtype = {"userId": np.int32, "movieId": np.int32, "rating": np.float32}
    # 加载数据，我们只用前三列数据，分别是用户ID，电影ID，已经用户对电影的对应评分
    ratings = pd.read_csv(data_path, dtype=dtype, usecols=range(3))
    testset_index = []
    for uid in ratings.groupby('userId').any().index:#上面有例子看any().index
        user_rating_data = ratings.where(ratings['userId'] == uid).dropna()  #相同保留，不同的nan
        if random:
            index = list(user_rating_data.index)  #这里是所有的
            np.random.shuffle(index) # shuffle洗牌 打乱顺序
            _index = round(len(user_rating_data)*x)  #拿80%作为训练集
            testset_index += list(index[_index:])  #剩余20%作为测试集
        else:
            index = round(len(user_rating_data)*x)
#             print(list(user_rating_data.index.values[index:]))
            testset_index += list(user_rating_data.index.values[index:])
        
    testset = ratings.loc[testset_index]
    trainset = ratings.drop(testset_index)
    print('完成数据集切分....')
    return trainset,testset

In [15]:
def accuray(predict_results, method="all"):
    '''
    准确性指标计算方法
    :param predict_results: 预测结果，类型为容器，每个元素是一个包含uid,iid,real_rating,pred_rating的序列
    :param method: 指标方法，类型为字符串，rmse或mae，否则返回两者rmse和mae
    :return:
    '''

    def rmse(predict_results):
        '''
        rmse评估指标
        :param predict_results:
        :return: rmse r是开方，m是平均，s是平方，e就是error
        '''
        length = 0
        _rmse_sum = 0
        for uid, iid, real_rating, pred_rating in predict_results:
            length += 1
            _rmse_sum += (pred_rating - real_rating) ** 2
        return round(np.sqrt(_rmse_sum / length), 4)

    def mae(predict_results):
        '''
        mae评估指标
        :param predict_results:
        :return: mae  mean abs error
        '''
        length = 0
        _mae_sum = 0
        for uid, iid, real_rating, pred_rating in predict_results:
            length += 1
            _mae_sum += abs(pred_rating - real_rating)
        return round(_mae_sum / length, 4)

    def rmse_mae(predict_results):
        '''
        rmse和mae评估指标
        :param predict_results:
        :return: rmse, mae
        '''
        length = 0
        _rmse_sum = 0
        _mae_sum = 0
        for uid, iid, real_rating, pred_rating in predict_results:
            length += 1
            _rmse_sum += (pred_rating - real_rating) ** 2
            _mae_sum += abs(pred_rating - real_rating)
        return round(np.sqrt(_rmse_sum / length), 4), round(_mae_sum / length, 4)

    if method.lower() == "rmse":
        return rmse(predict_results)
    elif method.lower() == "mae":
        return mae(predict_results)
    else:
        return rmse_mae(predict_results)

In [16]:
'''
LFM Model
'''
import pandas as pd
import numpy as np

# 评分预测    1-5
class LFM(object):

    def __init__(self, alpha, reg_p, reg_q, number_LatentFactors=10, number_epochs=10, columns=["uid", "iid", "rating"]):
        self.alpha = alpha # 学习率
        self.reg_p = reg_p    # P矩阵正则
        self.reg_q = reg_q    # Q矩阵正则
        self.number_LatentFactors = number_LatentFactors  # 隐式类别数量，就是那个k
        self.number_epochs = number_epochs    # 最大迭代次数
        self.columns = columns # 数据集的列名,默认为["uid", "iid", "rating"]

    def fit(self, dataset):
        '''
        fit dataset
        :param dataset: uid, iid, rating
        :return:
        '''

        self.dataset = pd.DataFrame(dataset)

        self.users_ratings = dataset.groupby(self.columns[0]).agg([list])[[self.columns[1], self.columns[2]]]
        self.items_ratings = dataset.groupby(self.columns[1]).agg([list])[[self.columns[0], self.columns[2]]]

        self.globalMean = self.dataset[self.columns[2]].mean()

        self.P, self.Q = self.sgd()

    def _init_matrix(self):
        '''
        初始化P和Q矩阵，同时为设置0，1之间的随机值作为初始值
        :return:
        '''
        # User-LF
        P = dict(zip(
            self.users_ratings.index,
            np.random.rand(len(self.users_ratings), self.number_LatentFactors).astype(np.float32)
        ))
        # Item-LF
        Q = dict(zip(
            self.items_ratings.index,
            np.random.rand(len(self.items_ratings), self.number_LatentFactors).astype(np.float32)
        ))
        return P, Q

    def sgd(self):
        '''
        使用随机梯度下降，优化结果
        :return:
        '''
        P, Q = self._init_matrix()

        for i in range(self.number_epochs):
            print("iter%d"%i)
            error_list = [] #为了计算均方误差
            for uid, iid, r_ui in self.dataset.itertuples(index=False):
                # User-LF P
                ## Item-LF Q
                v_pu = P[uid] #用户向量
                v_qi = Q[iid] #物品向量
                #np.dot(v_pu, v_qi)是两个矩阵相乘，得到预测值
                err = np.float32(r_ui - np.dot(v_pu, v_qi))
				#这里和上面的公式是完全对应的
                v_pu += self.alpha * (err * v_qi - self.reg_p * v_pu)
                v_qi += self.alpha * (err * v_pu - self.reg_q * v_qi)
                
                P[uid] = v_pu 
                Q[iid] = v_qi
				#注释是另外一种方式，逐个训练的，没那么容易理解
                # for k in range(self.number_of_LatentFactors):
                #     v_pu[k] += self.alpha*(err*v_qi[k] - self.reg_p*v_pu[k])
                #     v_qi[k] += self.alpha*(err*v_pu[k] - self.reg_q*v_qi[k])
				#把误差记录在列表中
                error_list.append(err ** 2)
            #打印误差的平均值
            print(np.sqrt(np.mean(error_list)))
        return P, Q

    def predict(self, uid, iid):
        # 如果uid或iid不在，我们使用全剧平均分作为预测结果返回
        if uid not in self.users_ratings.index or iid not in self.items_ratings.index:
            return self.globalMean

        p_u = self.P[uid]
        q_i = self.Q[iid]

        return np.dot(p_u, q_i)

    def test(self,testset):
        '''预测测试集数据'''
        for uid, iid, real_rating in testset.itertuples(index=False):
            try:
                pred_rating = self.predict(uid, iid)
            except Exception as e:
                print(e)
            else:
                yield uid, iid, real_rating, pred_rating

if __name__ == '__main__':
    dtype = [("userId", np.int32), ("movieId", np.int32), ("rating", np.float32)]
    dataset = pd.read_csv("../datasets/ml-latest-small/ratings.csv", usecols=range(3), dtype=dict(dtype))

    lfm = LFM(0.02, 0.01, 0.01, 10, 20, ["userId", "movieId", "rating"])
    lfm.fit(dataset)

    # while True:
    #     uid = input("uid: ")
    #     iid = input("iid: ")
    #     print(lfm.predict(int(uid), int(iid)))


iter0
0.9850127617957817
iter1
0.8797646247254687
iter2
0.8477038146339536
iter3
0.8287596100040213
iter4
0.8129003565990706
iter5
0.7977270478567604
iter6
0.7825865417942638
iter7
0.7676083500853487
iter8
0.7531908564072282
iter9
0.7396532857812655
iter10
0.7271401864774812
iter11
0.7156718549243851
iter12
0.7052034044965055
iter13
0.6956611447958873
iter14
0.6869626590616343
iter15
0.6790266517126993
iter16
0.6717769257270204
iter17
0.6651436293089248
iter18
0.6590633747864366
iter19
0.6534790071199813


In [17]:
#计算损失的代码

trainset, testset = data_split("../datasets/ml-latest-small/ratings.csv", random=True)

lfm = LFM(0.02, 0.01, 0.01, 10, 20, ["userId", "movieId", "rating"])
lfm.fit(trainset)
pred_results = lfm.test(testset)

rmse, mae = accuray(pred_results)

print("rmse: ", rmse, "mae: ", mae)

开始切分数据集...
完成数据集切分....
iter0
1.0063397059842487
iter1
0.8867495424560569
iter2
0.8499815207173984
iter3
0.8285910532670248
iter4
0.8112197432161979
iter5
0.7951065137990193
iter6
0.7793214182228966
iter7
0.7637068297668972
iter8
0.7484539332964452
iter9
0.733843879449497
iter10
0.7200999242587887
iter11
0.7073369702103457
iter12
0.6955758782710213
iter13
0.6847806054506901
iter14
0.6748893098996982
iter15
0.6658317374200634
iter16
0.6575366270456359
iter17
0.6499345055948671
iter18
0.6429589508517553
iter19
0.636547534916738
rmse:  1.0034 mae:  0.7667
